In [90]:
import re
import time
import json
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

from collections import defaultdict, Counter

from bs4 import BeautifulSoup
from requests_html import HTML

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Base de Bogota mi ciudad

In [4]:
url_cc = "https://www.bogotamiciudad.com/directoriodebogota/comercio/5-9/bogota/centros_comerciales#res"

In [7]:
tags = ["numero-resultado","list-negocio-nombre","list-direccion"]
data = []

for link in range(1,16,1):
    time.sleep(1)
    if link==1:
        req = requests.get(url_cc)
        html_str = req.text
        soup = BeautifulSoup(html_str, "html.parser")
    else:   
        req = requests.get(f"https://www.bogotamiciudad.com/directoriodebogota/comercio/5-9/bogota/centros_comerciales?p={link}#res")
        html_str = req.text
        soup = BeautifulSoup(html_str, "html.parser")

    row_data = defaultdict(list)
    for tag in tags:
        for row in soup.find_all(['span','h3','p'],attrs={"class" : tag}):
            row_data[tag].append(row.text)        
    data.append(dict(row_data))

dict_final = {}
for k in data[0].keys():
    dict_final[k] = sum([dict_final[k] for dict_final in data], [])

In [9]:
base = pd.DataFrame(dict_final)

In [10]:
base

,numero-resultado,list-negocio-nombre,list-direccion
0,1.,\n1.\nAndino\n,\n\r\nCarrera 11 #82-71 \r\...
1,2.,\n2.\nAntares\n,\n\r\nAutopista Sur #Cra 4 # 31-40 ...
2,3.,\n3.\nApolo\n,\n\r\nCalle 16 #8-95 \r\n ...
3,4.,\n4.\nArtesanías de la 53\n,\n\r\nCalle 53 #18-31 \r\n ...
4,5.,\n5.\nAsturias de Oviedo\n,\n\r\nCarrera 10 #1-50 Vía Guaymaral ...
...,...,...,...
145,146.,\n146.\nUnicentro de Occidente\n,\n\r\nCarrera 111C #86-05 \...
146,147.,\n147.\nUnilago\n,\n\r\nCarrera 15 #78-33 \r\...
147,148.,\n148.\nUnilago\n,\n\r\nAvenida Carrera 15 #78-33 ...
148,149.,\n149.\nUnisur\n,\n\r\nCarrera 4 #28-10 \r\n...


In [75]:
# ajuste df final
df_final = base.copy()
df_final['list-negocio-nombre'] = df_final['list-negocio-nombre'].apply(lambda x: re.sub(r"\s+"," ",x))
df_final['list-direccion'] = df_final['list-direccion'].apply(lambda x: re.sub(r"\s+"," ",x))
df_final= df_final[['list-negocio-nombre','list-direccion']] 

In [76]:
df_final['conteo'] = df_final['list-negocio-nombre'].apply(lambda x: x.count('.'))

In [77]:

df_final['list-negocio-nombre'] = np.where(df_final['list-negocio-nombre']==' 90. Mundo G.S.M ','90. Mundo Tech',df_final['list-negocio-nombre'])

In [78]:
df_final['list-negocio-nombre'][89]

'90. Mundo Tech'

In [79]:
df_final[['list-resultado','list-negocio-nombre']] = df_final['list-negocio-nombre'].str.split(".",expand = True)

In [80]:
df_final[['list-direccion','list-barrio']] = df_final['list-direccion'].str.split(" • ",expand = True)

In [81]:
df_final.drop(columns='conteo',inplace=True)

In [82]:
df_final = df_final[['list-resultado','list-negocio-nombre','list-direccion','list-barrio']]

In [84]:
for i in df_final.columns:
    df_final[i] = df_final[i].apply(lambda x: x.strip())

In [133]:
geolocator = Nominatim(timeout=10,user_agent="Pruebas")

In [134]:
#from geopy.extra.rate_limiter import RateLimiter
#geocode = RateLimiter(geolocator.geocode,min_delay_seconds=2)

In [136]:
df_final['conteo'] = df_final['list-direccion'].apply(lambda x: x.count('#'))

In [138]:
df_final[df_final["conteo"]>1]["list-direccion"]

1    Autopista Sur #Cra 4 # 31-40
Name: list-direccion, dtype: object

In [139]:
string_error = 'Autopista Sur #Cra 4 # 31-40'
df_final['list-direccion'] = np.where(df_final['list-direccion']==string_error,'Autopista Sur Cra 4 # 31-40',df_final['list-direccion'])

In [140]:
df_final[df_final["conteo"]>1]["list-direccion"]

1    Autopista Sur Cra 4 # 31-40
Name: list-direccion, dtype: object

In [141]:
df_final["full-direccion"] = df_final['list-direccion']+",Bogotá,Colombia"

In [ ]:
df_final["loc"] = df_final['full-direccion'].apply(geolocator.geocode)

In [143]:
df_final.drop(columns='conteo', inplace=True)

In [154]:
df_final["point"]= df_final["loc"].apply(lambda loc: tuple(loc.point) if loc else None)

In [160]:
df_final[['lat', 'lon', 'altitude']] = pd.DataFrame(df_final['point'].to_list(), index=df_final.index)

In [161]:
df_final

,list-resultado,list-negocio-nombre,list-direccion,list-barrio,loc,full-direccion,point,lat,lon,altitude
0,1,Andino,Carrera 11 #82-71,Chapinero,"(Carrera 11, Sosiego Sur, UPZ San José, Locali...","Carrera 11 #82-71,Bogotá,Colombia","(4.5755692, -74.0986757, 0.0)",4.575569,-74.098676,0.0
1,2,Antares,Autopista Sur Cra 4 # 31-40,Municipios Aledaños,None,"Autopista Sur Cra 4 # 31-40,Bogotá,Colombia",None,NaN,NaN,NaN
2,3,Apolo,Calle 16 #8-95,Santa Fé,"(Calle 16, El Vergel, UPZ Castilla, Localidad ...","Calle 16 #8-95,Bogotá,Colombia","(4.6547368, -74.1361171, 0.0)",4.654737,-74.136117,0.0
3,4,Artesanías de la 53,Calle 53 #18-31,Teusaquillo,None,"Calle 53 #18-31,Bogotá,Colombia",None,NaN,NaN,NaN
4,5,Asturias de Oviedo,Carrera 10 #1-50 Vía Guaymaral,Municipios Aledaños,None,"Carrera 10 #1-50 Vía Guaymaral,Bogotá,Colombia",None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
145,146,Unicentro de Occidente,Carrera 111C #86-05,Engativá,"(Carrera 111C, San Antonio Urbano, UPZ Engativ...","Carrera 111C #86-05,Bogotá,Colombia","(4.702363, -74.1343859, 0.0)",4.702363,-74.134386,0.0
146,147,Unilago,Carrera 15 #78-33,Chapinero,"(Carrera 15, San Antonio, UPZ Restrepo, Locali...","Carrera 15 #78-33,Bogotá,Colombia","(4.5885666, -74.0945999, 0.0)",4.588567,-74.094600,0.0
147,148,Unilago,Avenida Carrera 15 #78-33,Chapinero,"(Avenida Carrera 15, La Ermita, UPZ Country Cl...","Avenida Carrera 15 #78-33,Bogotá,Colombia","(4.7042454, -74.0426802, 0.0)",4.704245,-74.042680,0.0
148,149,Unisur,Carrera 4 #28-10,Municipios Aledaños,"(Carrera 4, Buenos Aires, UPZ Sosiego, Localid...","Carrera 4 #28-10,Bogotá,Colombia","(4.5799733, -74.0826426, 0.0)",4.579973,-74.082643,0.0


In [164]:
location = "Centro Comercial Andino, Chapinero"
geolocator.geocode(location)

Location(Centro Comercial Andino, UPZ Chicó Lago, Localidad Chapinero, Bogotá, Bogotá Distrito Capital, Región Andina, 11001, Colombia, (4.6675597, -74.05273652554914, 0.0))

# Prueba dos con el nombre del centro comercial

In [165]:
df_prueba2 = df_final.copy()
df_prueba2 = df_prueba2[["list-resultado",'list-negocio-nombre','list-barrio']]

In [170]:
df_prueba2['Nombre-CC'] = "Centro Comercial "+df_prueba2['list-negocio-nombre']+", "+'Colombia'
df_prueba2['loc'] = df_prueba2["Nombre-CC"].apply(geolocator.geocode)

In [ ]:
df_prueba2["point"]= df_prueba2["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
df_prueba2[['lat', 'lon', 'altitude']] = pd.DataFrame(df_prueba2['point'].to_list(), index=df_prueba2.index)

In [172]:
df_prueba2['list-barrio'].unique()

array(['Chapinero', 'Municipios Aledaños', 'Santa Fé', 'Teusaquillo',
       'Suba', 'San Cristóbal', 'Kennedy', 'Usaquén', 'Puente Aranda',
       'Candelaria', 'Antonio Nariño', 'Tunjuelito', 'Engativá',
       'Mártires', 'Fontibón', 'Barrios Unidos'], dtype=object)

# Otra pagina web

In [189]:
with open('../data/cc_bogota.json', encoding='utf-8') as f:
    cc_bogota = json.load(f)

In [190]:
df = pd.json_normalize(cc_bogota['features'])

In [193]:
df

,type,properties.CCENNOM,properties.CCENDIR,properties.CCENLOC,properties.CCENBARRIO,properties.CCENHOR,properties.CCENPROSER,properties.CCENNUMLOC,properties.CCENSITWEB,properties.LATITUD,properties.LONGITUD,geometry.type,geometry.coordinates
0,Feature,Pasaje Cultural y Comercial CED Centenario,Transversal 21 A # 21A - 65 Sur,Antonio Nariño,Restrepo - U.R. Alfonso Lopez Pumarejo,Lunes - Domingo 8:00 AM a 8:00 PM,"ropa, frutas y verduras, hiervas, bisutería.",153,http://www.ipes.gov.co,4.584853,-74.104596,Point,"[-74.10441164628877, 4.584759040295161]"
1,Feature,Caseta Feria Popular Rotonda Barrios Unidos,Calle 72 A # 67 - 75,Barrios Unidos,San Fernando,Lunes - Sábado 7:00 AM a 6:00 PM,"confitería, libros, servicio de arreglo de zap...",12,http://www.ipes.gov.co,4.677590,-74.085569,Point,"[-74.08559712975556, 4.677652125439473]"
2,Feature,Punto Comercial Siete de agosto,Carrera 25 # 66 - 65,Barrios Unidos,Siete De Agosto,Lunes - Sábado 7:00 AM a 7:00 PM,"ropa, marroquinería",29,http://www.ipes.gov.co,4.658268,-74.070989,Point,"[-74.07101805839586, 4.658329544591293]"
3,Feature,Plataforma Logística de Bosa,Calle 69 SUR # 95-56,Bosa,S.C. San Bernardino Xvi,Lunes - Domingo 7:00 AM a 7:00 PM,"bisutería, confitería y alimentos envasados",113,http://www.ipes.gov.co,4.632020,-74.199101,Point,"[-74.19912934060144, 4.632081462060175]"
4,Feature,Centro Comercial Caravana,Calle 12 # 9 - 68,Candelaria,La Catedral,Lunes - Sábado 7:00 AM a 7:00 PM Domingos - Fe...,"inciensos, infusiones de hierbas, cremas, jabo...",13,http://www.ipes.gov.co,4.600855,-74.077001,Point,"[-74.07702963098959, 4.600916373570125]"
5,Feature,Box Coulvert,Calle 12 # 9 - 68,Candelaria,La Catedral,Lunes - Sábado 7:00 AM a 7:00 PM Domingos - Fe...,"ropa, artesanías, productos naturistas como: p...",25,http://www.ipes.gov.co,4.600855,-74.077001,Point,"[-74.07702963098959, 4.600916373570125]"
6,Feature,Caseta Feria Popular Rotonda Candelaria,Calle 12 # 6 - 56,Candelaria,La Catedral,Lunes - Sábado 7:00 AM a 7:00 PM Domingos - Fe...,"servicio de cafetería, artesanías, joyería, se...",18,http://www.ipes.gov.co,4.598975,-74.074328,Point,"[-74.07435664804761, 4.599037235258418]"
7,Feature,Plaza Comercial y Artesanal Furatena,Calle 14 # 8-53,Candelaria,La Catedral,Lunes - Sábado 7:00 AM a 7:00 PM,"ropa, artesanías, productos naturales, frutas ...",81,http://www.ipes.gov.co,4.601595,-74.075275,Point,"[-74.07530332054178, 4.601657168931698]"
8,Feature,Caseta Feria Popular Rotonda Chapinero,Avenida Caracas # 50 - 60,Chapinero,S.C. Marly,Lunes - Sábado 7:00 AM a 7:00 PM,"servicio de arreglo de ropa, libros, artículos...",35,http://www.ipes.gov.co,4.637407,-74.066769,Point,"[-74.06679733211826, 4.637468524142977]"
9,Feature,Punto Comercial Flores de la 68,Avenida Carrera 13 # 68 - 76,Chapinero,La Esperanza Nororiental,Lunes - Domingo 7:00 AM a 7:00 PM,arreglos florales,44,http://www.ipes.gov.co,4.650404,-74.063202,Point,"[-74.06323055331636, 4.650465866025476]"


In [194]:
cc_bogota.keys()

dict_keys(['type', 'name', 'crs', 'features'])

In [197]:
cc_bogota['crs']

{'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4686'}}

# Valor del metro cuadrado de catastro

In [3]:
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
catastro = gpd.read_file('data_m2_catastro/data.geojson')